In [1]:
from ultralytics import YOLO

# Create a new YOLO model from scratch
model = YOLO('yolov8n.yaml')

# Load a pretrained YOLO model (recommended for training)
model = YOLO('yolov8n.pt')

# Train the model using the 'coco128.yaml' dataset for 3 epochs
results = model.train(data='coco128.yaml', epochs=3)

# Evaluate the model's performance on the validation set
results = model.val()

# Perform object detection on an image using the model
results = model('https://ultralytics.com/images/bus.jpg')

# Export the model to ONNX format
success = model.export(format='onnx')


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [28]:
import os
import shutil

parent_path = './data/DataCoffee_2022'
for i in os.listdir(parent_path):
    sub_path = parent_path + '/' + i
    for j in os.listdir(sub_path):
        path = sub_path + '/' + j
        for k in os.listdir(path):
            path_ = path + '/' + k
            total = len(os.listdir(path_))            
            n_i = 0
            n_x = 0            
            for l in os.listdir(path_):                   
                if l != '.ipynb_checkpoints':
                    if k == 'xml':
                        if n_x <= round(total * 0.8):                            
                            shutil.copy(path_ + '/' + l, './data/train/labels/' + l)
                        else:                            
                            # print(path_ + '/' + l)
                            shutil.copy(path_ + '/' + l, './data/valid/labels/' + l)
                        n_x = n_x + 1
                    elif k == 'img':
                        if n_i <= round(total * 0.8):                            
                            shutil.copy(path_ + '/' + l, './data/train/images/' + l)
                        else:
                            # print(path_ + '/' + l)
                            shutil.copy(path_ + '/' + l, './data/valid/images/' + l)
                        n_i = n_i + 1
    

In [40]:
#!pip install -U ultralytics
# %pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.8.18 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12037MiB)
Setup complete ✅ (16 CPUs, 94.2 GB RAM, 1830.0/1833.7 GB disk)


In [42]:
import yaml

data_yaml = dict(
    train ='train',
    val ='valid',    
    nc =10,
    names =['BLACK','BROKEN','BROWN','BigBroken','IMMATURE','INSECT','MOLD','PartlyBlack','LIGHTFM','HEAVYFM']
)

with open('coffee_data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)
    
%cat coffee_data.yaml

{names: [BLACK, BROKEN, BROWN, BigBroken, IMMATURE, INSECT, MOLD, PartlyBlack, LIGHTFM,
    HEAVYFM], nc: 10, train: train, val: valid}


In [43]:
names =['BLACK','BROKEN','BROWN','BigBroken','IMMATURE','INSECT','MOLD','PartlyBlack','LIGHTFM','HEAVYFM']
M=list(range(len(names)))
class_map=dict(zip(M,names))

In [56]:
model = YOLO("yolov8n.pt") 
model.to('cuda')

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [50]:
# !pip install xmltodict
import os
import xmltodict

def convert_coco_to_yolo(coco_xml_dir, yolo_txt_dir, class_mapping):
    for filename in os.listdir(coco_xml_dir):
        if filename.endswith(".xml"):
            with open(os.path.join(coco_xml_dir, filename), 'r') as file:
                data = xmltodict.parse(file.read())

            img_width = int(data['annotation']['size']['width'])
            img_height = int(data['annotation']['size']['height'])

            yolo_txt_path = os.path.join(yolo_txt_dir, filename.replace(".xml", ".txt"))

            with open(yolo_txt_path, 'w') as file:
                for obj in data['annotation']['object']:
                    class_name = obj['name']
                    class_id = class_mapping.get(class_name)
                    
                    if class_id is not None:
                        x_min = int(obj['bndbox']['xmin'])
                        y_min = int(obj['bndbox']['ymin'])
                        x_max = int(obj['bndbox']['xmax'])
                        y_max = int(obj['bndbox']['ymax'])

                        x_center = (x_min + x_max) / 2 / img_width
                        y_center = (y_min + y_max) / 2 / img_height
                        width = (x_max - x_min) / img_width
                        height = (y_max - y_min) / img_height

                        file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

# Example usage
name = ['BLACK','BROKEN','BROWN','BigBroken','IMMATURE','INSECT','MOLD','PartlyBlack','LIGHTFM','HEAVYFM']
class_mapping = {
    'BLACK': 0,
    'BROKEN': 1,
    'BROWN': 2,
    'BigBroken': 3,
    'IMMATURE': 4,
    'INSECT': 5,
    'MOLD': 6,
    'PartlyBlack': 7,
    'LIGHTFM': 8,
    'HEAVYFM': 9,
    # Add more class mappings as needed
}

coco_xml_dir = './datasets/valid/labels'
yolo_txt_dir = './datasets/valid/labels_txt'

convert_coco_to_yolo(coco_xml_dir, yolo_txt_dir, class_mapping)

In [62]:
from ultralytics import settings

# Update a setting
settings.update({'runs_dir': '.'})

# Update multiple settings
# settings.update({'runs_dir': '/path/to/runs', 'tensorboard': False})
# Reset settings to default values
settings.reset()

model = YOLO("yolov8n.pt") 
model.to('cuda')

!yolo task=detect mode=train model=yolov8n.pt device='cuda' data=coffee_data.yaml epochs=300 imgsz=1280 batch=4 cache=true

Ultralytics YOLOv8.0.199 🚀 Python-3.8.18 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12037MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coffee_data.yaml, epochs=300, patience=50, batch=4, imgsz=1280, save=True, save_period=-1, cache=True, device=cuda, workers=8, project=None, name=train19, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, o

In [39]:
from ultralytics import settings

# Update a setting
settings.update({'runs_dir': '.'})

# Update multiple settings
# settings.update({'runs_dir': '/path/to/runs', 'tensorboard': False})

# Reset settings to default values
settings.reset()

!yolo train data=coffee.yaml model=yolov8n.pt epochs=3 imgsz=1280 batch=16

Ultralytics YOLOv8.0.199 🚀 Python-3.8.18 torch-2.1.0 CPU (AMD Ryzen 7 5800X 8-Core Processor)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coffee.yaml, epochs=3, patience=50, batch=16, imgsz=1280, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=Fal

In [7]:
license_results = [('unknown', ('plate', 0.9952951073646545, (243, 586, 279, 622), 10600, 0.9433962264150944, (24, 391, 440, 807))), ('58Y1847957', ('plate', 0.9958018660545349, (1152, 791, 1199, 835), 20825, 0.68, (576, 156, 1344, 924)))]
print(license_results)

for i in license_results:
    if list(i)[0] != 'unknown':
        # print(list(i)[0])
        plate_count = (
            Plate.select(Plate.text)
            .where( Plate.text == list(i)[0] )
            .count()
        )
        
        if plate_count > 0:
            now = datetime.datetime.now()
            selected_columns = [
                Event.id,
                Event.camera,
                Event.label,
                Event.zones,
                Event.start_time,
                Event.end_time,
                Event.has_clip,
                Event.has_snapshot,
                Event.plus_id,
                Event.retain_indefinitely,
                Event.sub_label,
                Event.top_score,
                Event.false_positive,
                Event.box,
                Event.data,
            ]

            # clauses.append((start_hour_fun > time_after))
            # clauses.append((start_hour_fun < time_before))

            events = (
                Event.select(*selected_columns)
                .where(Event.data["attributes"][0]["text"] == id)
                .order_by(Event.start_time.desc())
                .limit(1)
                .dicts()
            )
            resp = requests.get(f"{FRIGATE_LOCALHOST}/api/send/email",params={'plate': list(i)[0], 'time': now.strftime("%Y-%m-%d %H:%M:%S")})
            
            


[('unknown', ('plate', 0.9952951073646545, (243, 586, 279, 622), 10600, 0.9433962264150944, (24, 391, 440, 807))), ('58Y1847957', ('plate', 0.9958018660545349, (1152, 791, 1199, 835), 20825, 0.68, (576, 156, 1344, 924)))]
58Y1847957
